In [2]:
# default_exp run_flow

In [3]:
%load_ext autoreload
%autoreload 2

In [48]:
# export


import asyncio
import multiprocessing
import os
import sys
from itertools import product
from pathlib import Path
from typing import Any, Dict, Iterable

import pandas as pd
from fastcore.script import call_parse
from nbdev.export import find_default_export, get_config, read_nb

from sciflow.utils import chunks, get_flow_path, prepare_env, run_shell_cmd

# Verify and Run Sciflow Flows

In [5]:
nb_path = Path(Path(".").resolve(), "test", "test_export.ipynb")
flow_path = get_flow_path(nb_path, flow_provider="sagemaker")
nb = read_nb(nb_path)
module_name = find_default_export(nb["cells"]).replace(".", "/")
test_module = os.path.join(get_config().path("lib_path"), f"{module_name}.py")
flows_dir = get_config(cfg_name="test/settings.ini").path("flows_path")
flow_name = os.path.basename(test_module)

# Verify or Run an Individual Flow

> `subprocess` is used to run flows as most flow providers bundle a CLI which makes for a consistent execution experience with minimal adaptation.

In [6]:
# export


def check_is_init():
    root_path = str(get_config().path("root_path"))

    if root_path not in sys.path:
        print(f"PYTHONPATH={sys.path}")
        raise ValueError("Project is not in path; have you run sciflow_init?")

In [7]:
# export


def make_shell_cmd(
    flow_nb_path, flow_provider="metaflow", flow_command="show", params=None
):
    prepare_env()
    if flow_nb_path.suffix == ".ipynb":
        flow_path = get_flow_path(flow_nb_path, flow_provider=flow_provider)
    else:
        flow_path = flow_nb_path
    if params:
        args = " ".join([f"--{k} {v}" for k, v in params.items()])

        flow_command = f"{flow_command} {args}"

    return f"python '{flow_path}' {flow_command}"

In [8]:
# export


def check_call_flow(
    flow_nb_path, flow_provider="metaflow", flow_command="show", params=None
):
    check_is_init()

    cmd = make_shell_cmd(flow_nb_path, flow_provider, flow_command, params)
    pipe, output = run_shell_cmd(cmd)
    return pipe.returncode, output

In [9]:
sys_path = sys.path
sys.path = []

In [10]:
raised = False
try:
    check_call_flow(nb_path, flow_provider="metaflow", flow_command="show")
except:
    raised = True
assert raised

In [11]:
sys.path = sys_path

In [12]:
prepare_env()

In [13]:
ret_code, output = check_call_flow(
    nb_path, flow_provider="metaflow", flow_command="show"
)
assert ret_code == 0

In [14]:
# slow

ret_code, output = check_call_flow(
    nb_path, flow_provider="metaflow", flow_command="run"
)
assert ret_code == 0
print(output)

Started tracking flow: flow-2022-10-21-16-02-47-854
Metaflow 2.5.2 executing TestExportFlow for user:'Donal Simmie'
Validating your flow...
    The graph looks good!
2022-10-21 16:02:54.944 Workflow starting (run-id 1666368174513434):
2022-10-21 16:02:55.026 [1666368174513434/start/1 (pid 30465)] Task is starting.
2022-10-21 16:03:01.305 [1666368174513434/start/1 (pid 30465)] Starting Heartbeat
2022-10-21 16:03:01.323 [1666368174513434/start/1 (pid 30465)] 3
2022-10-21 16:03:02.144 [1666368174513434/start/1 (pid 30465)] Stopping Heartbeat
2022-10-21 16:03:02.177 [1666368174513434/start/1 (pid 30465)] Task finished successfully.
2022-10-21 16:03:02.290 [1666368174513434/preprocess/2 (pid 30479)] Task is starting.
2022-10-21 16:03:08.565 [1666368174513434/preprocess/2 (pid 30479)] Starting Heartbeat
2022-10-21 16:03:08.571 [1666368174513434/preprocess/2 (pid 30479)] Preprocessing input data from /home/sagemaker-user/git/sciflow/nbs...
2022-10-21 16:03:09.572 [1666368174513434/preprocess/

In [15]:
# slow

ret_code, output = check_call_flow(nb_path, flow_command="run")
assert ret_code == 0
print(output)

Started tracking flow: flow-2022-10-21-16-03-28-978
Metaflow 2.5.2 executing TestExportFlow for user:'Donal Simmie'
Validating your flow...
    The graph looks good!
2022-10-21 16:03:36.124 Workflow starting (run-id 1666368215706522):
2022-10-21 16:03:36.202 [1666368215706522/start/1 (pid 30696)] Task is starting.
2022-10-21 16:03:42.632 [1666368215706522/start/1 (pid 30696)] Starting Heartbeat
2022-10-21 16:03:42.635 [1666368215706522/start/1 (pid 30696)] 3
2022-10-21 16:03:43.382 [1666368215706522/start/1 (pid 30696)] Stopping Heartbeat
2022-10-21 16:03:43.410 [1666368215706522/start/1 (pid 30696)] Task finished successfully.
2022-10-21 16:03:43.515 [1666368215706522/preprocess/2 (pid 30709)] Task is starting.
2022-10-21 16:03:49.821 [1666368215706522/preprocess/2 (pid 30709)] Starting Heartbeat
2022-10-21 16:03:49.824 [1666368215706522/preprocess/2 (pid 30709)] Preprocessing input data from /home/sagemaker-user/git/sciflow/nbs...
2022-10-21 16:03:50.825 [1666368215706522/preprocess/

In [16]:
ret_code, output = check_call_flow(
    nb_path, flow_provider="sagemaker", flow_command="show"
)
assert ret_code == 0

In [17]:
print(output)

Sciflow generated pipeline is valid
Pipeline name: test-export
Pipeline ARN: arn:aws:sagemaker:eu-west-1:368653567616:pipeline/test-export


In [18]:
# slow

ret_code, output = check_call_flow(
    nb_path, flow_provider="sagemaker", flow_command="run"
)
print(output)
assert ret_code == 0

Started tracking flow: pipeline-2022-10-21-16-04-24-824
Starting Sciflow generated pipeline: pipeline-2022-10-21-16-04-24-824
{'PipelineArn': 'arn:aws:sagemaker:eu-west-1:368653567616:pipeline/test-export', 'PipelineExecutionArn': 'arn:aws:sagemaker:eu-west-1:368653567616:pipeline/test-export/execution/5ushkppa2dqf', 'PipelineExecutionDisplayName': 'execution-1666368269400', 'PipelineExecutionStatus': 'Executing', 'CreationTime': datetime.datetime(2022, 10, 21, 16, 4, 29, 273000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2022, 10, 21, 16, 4, 29, 273000, tzinfo=tzlocal()), 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:eu-west-1:368653567616:user-profile/d-likrmmebxomz/donal', 'UserProfileName': 'donal', 'DomainId': 'd-likrmmebxomz'}, 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:eu-west-1:368653567616:user-profile/d-likrmmebxomz/donal', 'UserProfileName': 'donal', 'DomainId': 'd-likrmmebxomz'}, 'ResponseMetadata': {'RequestId': 'abc39959-4eed-4c08-a003-f8aa

# Verify/Run all Flows

In [19]:
# export


def check_call_flows(
    config,
    flow_provider="metaflow",
    flow_command="show",
    ignore_suffix=None,
    exit_on_error=True,
):
    flow_results = {}
    flows_dir = Path(config.path("flows_path"), flow_provider)

    if ignore_suffix:
        flow_file_names = [
            p for p in os.listdir(flows_dir) if not p.endswith(ignore_suffix)
        ]
    else:
        flow_file_names = os.listdir(flows_dir)
    ret_codes = []
    exit_code = 0
    for flow_file_name in flow_file_names:
        flow_name = os.path.basename(flow_file_name)
        if flow_file_name.startswith("_sciflow"):
            continue
        if flow_file_name.endswith(".py"):
            ret_code, output = check_call_flow(
                Path(flows_dir, flow_file_name), flow_command=flow_command
            )
            flow_results[flow_name] = ret_code, output
            if ret_code == 0:
                print(f"Flow: {flow_name} {flow_command} verified")
            else:
                print(
                    f"Flow: {flow_name} {flow_command} verification failed\nDetails:\n{output}"
                )
            ret_codes.append(ret_code)
    if any([rc != 0 for rc in ret_codes]):
        exit_code = 1
        try:
            # Exit with an error code if running from a non interactive Python environment.
            get_ipython().__class__.__name__
        except NameError:
            if exit_on_error:
                return sys.exit(exit_code)
    return exit_code

In [20]:
check_call_flows(get_config(cfg_name="test/settings.ini"))

Flow: test_export.py show verified
Flow: test_data_handling.py show verified
Flow: test_module.py show verified
Flow: test_multistep_no_params.py show verified
Flow: test_tracking.py show verified
Flow: test_multistep.py show verified


0

In [21]:
# slow

check_call_flows(
    get_config(cfg_name="test/settings.ini"),
    flow_command="--no-pylint run",
    ignore_suffix="_no_params.py",
)

Flow: test_export.py --no-pylint run verified
Flow: test_data_handling.py --no-pylint run verified
Flow: test_module.py --no-pylint run verified
Flow: test_tracking.py --no-pylint run verified
Flow: test_multistep.py --no-pylint run verified


0

In [22]:
check_call_flows(get_config(cfg_name="test/settings.ini"), flow_provider="sagemaker")

Flow: test_export.py show verified
Flow: test_data_handling.py show verified
Flow: test_module.py show verified
Flow: test_multistep_no_params.py show verified
Flow: test_tracking.py show verified
Flow: test_multistep.py show verified


0

In [23]:
# slow

nb_path = Path(Path(".").resolve(), "test", "test_multistep.ipynb")
ret_code, output = check_call_flow(
    nb_path,
    flow_command="run",
    params={"traffic_percent": 1, "model_level": "dispatcher"},
)
print(output)
assert ret_code == 0

Started tracking flow: flow-2022-10-21-16-10-48-927
Metaflow 2.5.2 executing TestMultistepFlow for user:'Donal Simmie'
Validating your flow...
    The graph looks good!
2022-10-21 16:10:52.132 Workflow starting (run-id 1666368651634037):
2022-10-21 16:10:52.232 [1666368651634037/start/1 (pid 31363)] Task is starting.
2022-10-21 16:10:54.595 [1666368651634037/start/1 (pid 31363)] Starting Heartbeat
2022-10-21 16:10:54.596 [1666368651634037/start/1 (pid 31363)] The first step
2022-10-21 16:10:55.438 [1666368651634037/start/1 (pid 31363)] Stopping Heartbeat
2022-10-21 16:10:55.469 [1666368651634037/start/1 (pid 31363)] Task finished successfully.
2022-10-21 16:10:55.583 [1666368651634037/preprocess/2 (pid 31376)] Task is starting.
2022-10-21 16:10:57.897 [1666368651634037/preprocess/2 (pid 31376)] Starting Heartbeat
2022-10-21 16:10:57.920 [1666368651634037/preprocess/2 (pid 31376)] I captialised the message: THE FIRST STEP
2022-10-21 16:10:57.921 [1666368651634037/preprocess/2 (pid 31376

# Aynsc Flow Running

> Run the flow you are working on from the notebook you are working on. This maximises the amount of experiments you can run as you don't have down time. While long running tasks are running you can keep exploring! :-)

In [24]:
# export


async def flow_task(
    flow_nb_path, flow_provider="metaflow", flow_command="run", params=None
):
    cmd = make_shell_cmd(flow_nb_path, flow_provider, flow_command, params)

    proc = await asyncio.create_subprocess_shell(
        cmd, stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE
    )

    stdout, stderr = await proc.communicate()

    # print(f"[{cmd!r} exited with {proc.returncode}]")
    err = ""
    out = ""
    if stderr:
        err = f'[stderr]\n{stderr.decode("utf-8").strip()}'
    if stdout:
        out = f'[stdout]\n{stdout.decode("utf-8").strip()}'

    return proc.returncode, err + out

In [25]:
# export


def run_flow_async(
    flow_nb_path, flow_provider="metaflow", flow_command="run", params=None
):
    loop = asyncio.get_event_loop()
    task = loop.create_task(
        flow_task(flow_nb_path, flow_provider, flow_command, params)
    )
    return task

In [26]:
# slow

task = run_flow_async(
    Path(Path(".").resolve(), "test", "test_multistep.ipynb"),
    params={"traffic_percent": 10, "workers": 12},
)
task

<Task pending name='Task-9' coro=<flow_task() running at /tmp/ipykernel_30399/3195946906.py:4>>

In [27]:
# slow
await task
assert 0 == task.result()[0]

In [28]:
# slow

task = run_flow_async(
    Path(Path(".").resolve(), "test", "test_export.ipynb"),
    flow_provider="sagemaker",
    params={"some_param": "async"},
)
task

<Task pending name='Task-15' coro=<flow_task() running at /tmp/ipykernel_30399/3195946906.py:4>>

In [29]:
# slow

await task
assert 0 == task.result()[0]

In [30]:
# slow

task = run_flow_async(
    Path(Path(".").resolve(), "test", "test_multistep.ipynb"),
    flow_provider="sagemaker",
    params={"traffic_percent": 10, "workers": 12},
)
task

<Task pending name='Task-21' coro=<flow_task() running at /tmp/ipykernel_30399/3195946906.py:4>>

In [31]:
# slow

await task
assert 0 == task.result()[0]

In [32]:
# export


async def run_flows_async(
    config,
    flow_provider="metaflow",
    flow_command="run",
    params=None,
    ignore_suffix=None,
    exit_on_error=True,
):
    flow_tasks = {}
    flows_dir = Path(config.path("flows_path"), flow_provider)

    if ignore_suffix:
        flow_file_names = [
            p for p in os.listdir(flows_dir) if not p.endswith(ignore_suffix)
        ]
    else:
        flow_file_names = os.listdir(flows_dir)
    ret_codes = []
    exit_code = 0
    loop = asyncio.get_event_loop()

    for flow_file_name in flow_file_names:
        flow_name = os.path.basename(flow_file_name)
        if flow_file_name.startswith("_sciflow"):
            continue
        if flow_file_name.endswith(".py"):
            task = loop.create_task(
                flow_task(
                    Path(flows_dir, flow_file_name), flow_provider, flow_command, params
                )
            )
            flow_tasks[flow_name] = task

    for flow_name, task in flow_tasks.items():
        await task
        ret_code = task.result()[0]
        if ret_code == 0:
            print(f"Flow: {flow_name} {flow_command} verified")
        else:
            print(
                f"Flow: {flow_name} {flow_command} verification failed\nDetails:\n{output}"
            )
        ret_codes.append(ret_code)
    if any([rc != 0 for rc in ret_codes]):
        exit_code = 1
        try:
            # Exit with an error code if running from a non interactive Python environment.
            get_ipython().__class__.__name__
        except NameError:
            if exit_on_error:
                return sys.exit(exit_code)
    return exit_code

In [33]:
# slow

task = run_flows_async(
    get_config(cfg_name="test/settings.ini"),
    flow_command="--no-pylint run",
    ignore_suffix="_no_params.py",
)

In [34]:
# slow

await task

Flow: test_export.py --no-pylint run verified
Flow: test_data_handling.py --no-pylint run verified
Flow: test_module.py --no-pylint run verified
Flow: test_tracking.py --no-pylint run verified
Flow: test_multistep.py --no-pylint run verified


0

In [35]:
# slow

task = run_flows_async(
    get_config(cfg_name="test/settings.ini"),
    flow_provider="sagemaker",
    flow_command="run",
    ignore_suffix="_no_params.py",
)

In [36]:
# slow

await task

Flow: test_export.py run verified
Flow: test_data_handling.py run verified
Flow: test_module.py run verified
Flow: test_tracking.py run verified
Flow: test_multistep.py run verified


0

# Utilities to Search Parameter Space

In [37]:
param_grid = {
    "traffic_percent": [1, 5, 10, 20, 50, 100],
    "model_level": ["router", "dispatcher"],
    "workers": [1],
}

In [38]:
# export


def iter_param_grid(param_grid):
    # https://github.com/scikit-learn/scikit-learn/blob/main/sklearn/model_selection/_search.py
    for p in [param_grid]:
        # Always sort the keys of a dictionary, for reproducibility
        items = sorted(p.items())
        if not items:
            yield {}
        else:
            keys, values = zip(*items)
            for v in product(*values):
                params = dict(zip(keys, v))
                yield params

In [39]:
assert [{"a": 1, "b": 1, "c": "hello"}, {"a": 2, "b": 1, "c": "hello"}] == list(
    iter_param_grid({"a": [1, 2], "b": [1], "c": ["hello"]})
)

In [40]:
# export


def sample_grid_space(param_grid: Dict[str, Iterable[Any]], num_samples: int):
    samples = []
    for i, sample in enumerate(iter_param_grid(param_grid)):
        samples.append(sample)
    if num_samples < len(samples):
        samples = pd.Series(samples).sample(num_samples).tolist()
    return samples

In [41]:
sample_space = sample_grid_space({"a": [1, 2], "b": [1], "c": ["hello"]}, 1)
assert sample_space[0]["b"] == 1
assert sample_space[0]["c"] == "hello"
assert sample_space[0]["a"] == 1 or sample_space[0]["a"] == 2

In [42]:
# export


async def search_batches(flow_nb_path, flow_provider, task_batches):
    futures = []
    loop = asyncio.get_event_loop()
    for task_batch in task_batches:
        tasks = [
            (
                loop.create_task(
                    flow_task(
                        flow_nb_path,
                        flow_provider,
                        flow_command="run",
                        params=param_spec,
                    )
                )
            )
            for param_spec in task_batch
        ]
        futures.append(await asyncio.wait(tasks))
    return futures

In [43]:
# export


def search_flow_grid(
    flow_nb_path,
    param_grid,
    flow_provider="metaflow",
    total_tasks=None,
    n_conc_tasks=None,
    local_mode=True,
):
    if total_tasks is None:
        total_tasks = len(list(iter_param_grid(param_grid)))

    if local_mode and n_conc_tasks is None:
        n_conc_tasks = int((multiprocessing.cpu_count() / 2) - 1)

    sample_space = sample_grid_space(param_grid, total_tasks)
    task_batches = list(chunks(sample_space, n_conc_tasks))
    futures = search_batches(flow_nb_path, flow_provider, task_batches)
    return futures

In [44]:
# export


def extract_results(future_tasks):
    completed_tasks = [
        item for sublist in [list(ft[0]) for ft in future_tasks] for item in sublist
    ]
    results = [t.result() for t in completed_tasks]
    return results

In [45]:
nb_path = Path(
    Path(".").resolve(),
    "test",
    "test_export.ipynb",
)
param_grid = {
    "traffic_percent": [1, 2, 3, 4, 5, 7, 8, 10, 20, 30, 40, 50],
    "model_level": ["dispatcher"],
    "workers": [1],
}

In [ ]:
# slow

future_tasks = await ensure_future(
    search_flow_grid(
        nb_path,
        param_grid,
        flow_provider="sagemaker",
        total_tasks=None,
        n_conc_tasks=4,
    )
)

In [ ]:
# slow

results = extract_results(future_tasks)
results

In [ ]:
# export


@call_parse
def sciflow_check_metaflows():
    check_call_flows(get_config())

In [ ]:
# export


@call_parse
def sciflow_check_sagemaker_flows():
    check_call_flows(get_config(), flow_provider="sagemaker")

In [ ]:
# export


@call_parse
def sciflow_run_metaflows():
    check_call_flows(get_config(), flow_command="run")

In [ ]:
# export


@call_parse
def sciflow_run_sagemaker_flows():
    check_call_flows(get_config(), flow_command="run", flow_provider="sagemaker")